## Data cleaning

In [32]:
import csv
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.util import ngrams

with open('dara 1_train.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    """
    for row in reader:
        print(row['example_id'])
        print(row[' text'])
        print(row[' aspect_term'])
        print(row[' term_location'])
        print(row[' class'])
        print()        
    """
    happy_emoji = [':‑)',':)',':-]',':]',':-3',':3',':->',':>','8-)','8)',':-}', ':}',':o)',':c)',':^)','=]','=)',
                  ':‑D',':D','8‑D','8D','x‑D','xD','X‑D','XD','=D','=3','B^D',':-))']
                
    sad_emoji = [':‑(',':(',':‑c',':c',':‑<',':<',':‑[',':[',':-||','>:[',':{',':@','>:(']
    
    negative_words = ['doesn’t', 'isn’t', 'wasn’t', 'shouldn’t', 'wouldn’t', 'couldn’t', 'won’t', 'can’t', 'don’t']
    
    tokenizer = RegexpTokenizer(r'\w+')
    
    stop_words = set(stopwords.words('english'))
    
    singles = []
    texts = []
    tempText = ''
    
    for row in reader:
        tempText = row[' text'].replace('[comma]',',').lower()
               
        for hem in happy_emoji:
            if hem in tempText:
                tempText = tempText.replace(hem, 'emojihappy')
        
        for sem in sad_emoji:
            if sem in tempText:
                tempText = tempText.replace(sem, 'emojisad')
                
        for negw in negative_words:
            if negw in tempText:
                tempText = tempText.replace(negw, 'not')
        
        singles.append(tokenizer.tokenize(tempText))
        
        texts.append(tempText)
        
    newTokenList = []
    
    stemmer = SnowballStemmer("english")
    
    for tokenList in singles:
        newToken = []
        for token in tokenList:
            if token not in stop_words:
                newToken.append(stemmer.stem(token))
        newTokenList.append(newToken)
    
    unigrams = ngrams(token,1)

In [33]:
#experiment
'''
for text in texts:
    print('\n'+text)
'''
'''
for sss in singles:
    print(sss)
'''
'''
for tk in newTokenList:
    print(tk)
'''


['obvious', 'one', 'import', 'featur', 'comput', 'human', 'interfac']
['good', 'everi', 'day', 'comput', 'web', 'brows']
['keyboard', 'alright', 'plate', 'around', 'cheap', 'plastic', 'make', 'hollow', 'sound', 'use', 'mous', 'command', 'button']
['problem', 'right', 'speaker', 'work']
['problem', 'dell', 'custom', 'servic']
['partial', 'due', 'fact', 'run', 'window', 'vista']
['accord', 'decid', 'never', 'purchas', 'anoth', 'hp', 'product', 'five', 'year', 'old', 'compaq', 'last', '5', 'year', 'hard', 'drive', 'crash']
['think', 'camera', 'one', 'requir', 'forgot', 'check', 'specif', 'one', 'purchas']
['cord', 'come', 'right', 'power', 'plus', 'cord', 'come', 'front', 'headphon', 'mic', 'plus', 'network', 'connect', 'left', 'make', 'messi', 'setup', 'cord', 'go', 'everi', 'direct']
['recent', 'safari', 'internet', 'browser', 'freak', 'use', 'firefox', 'instead']
['last', 'comput', 'toshiba', 'cost', '400', 'work', 'like', 'charm', 'mani', 'year']
['machin', 'bluray', 'player', 'book',

['type', 'letter', 'space', 'frequent', 'omit', 'requir', 'user', 'redo', 'mani', 'word', 'sentenc']
['system', 'loos', '20', 'perform']
['say', 'enough', 'satisfi', 'product', 'help', 'aftermarket']
['_', 'board', 'bad', 'connector', 'power', 'suppli', 'short', 'warrenti', 'expir', 'power', 'suppli', 'start', 'issu']
['easi', 'use', 'good', 'qualiti', 'good', 'price']
['one', 'drawback', 'wish', 'key', 'backlit']
['screen', 'resolut', 'exact', 'look']
['within', '5', 'month', 'charger', 'crap']
['would', 'definit', 'reccomend', 'market', 'easi', 'use', 'stylish', 'fun', 'awesom', 'comput']
['absolut', 'expens', 'pc', 'laptop', 'eas', 'use', 'secur', 'minim', 'problem', 'arisen', 'make', 'well', 'worth', 'pricetag']
['comput', 'froze', 'sever', 'occas', 'button', 'random', 'would', 'fall', 'even', 'moment', 'comput', 'would', 'refus', 'turn']
['also', 'fun', 'use']
['rare', 'work', 'incred', 'slow']
['program', 'came', 'ilif', 'iwork', 'set', 'begin']
['stamp', 'piec', 'therefor', 'str

In [3]:
#experiment
exps = ['asd','asd_','_wqeqwe']
expsf = []
for ex in exps:
    expsf.append(ex.replace('_',''))
print(expsf)

['asd', 'asd', 'wqeqwe']


In [4]:
lhap = [':)',':P',':D']
lsad = [':(',':|']
t1 = "i am happy :), but he is sad :|"
t2 = "i am sad :(, he is :P"
t1.replace(lhap,'smilehappy')
t1.replace(lsad,'smilesad')
t2.replace(lhap,'smilehappy')
t2.replace(lsad,'smilesad')
print(t1)
print(t2)

TypeError: replace() argument 1 must be str, not list

In [21]:
#experiment

happy_emoji = [':‑)',':)',':-]',':]',':-3',':3',':->',':>','8-)','8)',':-}', ':}',':o)',':c)',':^)','=]','=)',
                  ':‑D',':D','8‑D','8D','x‑D','xD','X‑D','XD','=D','=3','B^D',':-))']
                
sad_emoji = [':‑(',':(',':‑c',':c',':‑<',':<',':‑[',':[',':-||','>:[',':{',':@','>:(']

strr = "i am happy :)"
for hem in happy_emoji:
    if hem in strr:
        strr = strr.replace(hem, 'emojihappy')

for sem in sad_emoji:
    strr.replace(sem, 'emojisad')
    
print(strr)


i am happy emojihappy


In [6]:
import pandas as pd

data = pd.read_csv('dara 1_train.csv')
print(data.shape)
data.head()


data2 = pd.read_csv('data 2_train.csv')
print(data2.shape)
data2.head()

(2203, 5)
(3602, 5)


,example_id,text,aspect_term,term_location,class
0,3121_0,But the staff was so horrible to us.,staff,8--13,-1
1,2777_0,To be completely fair[comma] the only redeemin...,food,57--61,1
2,1634_0,The food is uniformly exceptional[comma] with ...,food,4--8,1
3,1634_1,The food is uniformly exceptional[comma] with ...,kitchen,55--62,1
4,1634_2,The food is uniformly exceptional[comma] with ...,menu,141--145,0


In [3]:
data.describe()

,class
count,2203.000000
mean,0.050386
std,0.894378
min,-1.000000
25%,-1.000000
50%,0.000000
75%,1.000000
max,1.000000


In [4]:
data.iloc[:100,:]

,example_id,text,aspect_term,term_location,class
0,2333_1,Obviously one of the most important features o...,human interface,69--84,0
1,1805_1,Good for every day computing and web browsing.,every day computing,9--28,1
2,2782_2,while the keyboard itself is alright[comma] th...,mouse command buttons,115--136,-1
3,1385_0,Again[comma] the same problem[comma] the right...,right speaker,29--42,-1
4,1423_0,My problem was with DELL Customer Service.,DELL Customer Service,20--41,-1
5,104_0,Is this partially due to the fact that it is r...,Windows Vista,53--66,0
6,398_0,Accordingly[comma] I have decided to NEVER pur...,hard drive,117--127,-1
7,2766_1,I did think it had a camera because that was o...,specifications,96--110,0
8,2808_5,Cords coming out the right for power plus cord...,setup,138--143,-1
9,1727_0,The most recent being that my Safari internet ...,Safari internet browser,30--53,-1


In [12]:
data.iloc[:,:100]

,example_id,text,aspect_term,term_location,class
0,2333_1,Obviously one of the most important features o...,human interface,69--84,0
1,1805_1,Good for every day computing and web browsing.,every day computing,9--28,1
2,2782_2,while the keyboard itself is alright[comma] th...,mouse command buttons,115--136,-1
3,1385_0,Again[comma] the same problem[comma] the right...,right speaker,29--42,-1
4,1423_0,My problem was with DELL Customer Service.,DELL Customer Service,20--41,-1
5,104_0,Is this partially due to the fact that it is r...,Windows Vista,53--66,0
6,398_0,Accordingly[comma] I have decided to NEVER pur...,hard drive,117--127,-1
7,2766_1,I did think it had a camera because that was o...,specifications,96--110,0
8,2808_5,Cords coming out the right for power plus cord...,setup,138--143,-1
9,1727_0,The most recent being that my Safari internet ...,Safari internet browser,30--53,-1


In [8]:
pd.set_option('display.max_colwidth', -1)
data.loc[:,[' text']]

,text
0,"Obviously one of the most important features of any computer is the ""human interface."
1,Good for every day computing and web browsing.
2,while the keyboard itself is alright[comma] the plate around it is cheap plastic and makes a hollow sound when using the mouse command buttons.
3,Again[comma] the same problem[comma] the right speaker did not work.
4,My problem was with DELL Customer Service.
5,Is this partially due to the fact that it is running Windows Vista?
6,Accordingly[comma] I have decided to NEVER purchase another HP product (my five year old Compaq) lasted 5-years before the hard drive crashed.
7,I did think it had a camera because that was one of my requirements[comma] but forgot to check in the specifications on this one before I purchased.
8,Cords coming out the right for power plus cords coming out front for headphones/mic plus network connection on left make for a very messy setup with cords going every direction.
9,The most recent being that my Safari internet browser is freaking out on me[comma] but I have just been using firefox instead.


In [18]:
data.replace("[comma]",",")

,example_id,text,aspect_term,term_location,class
0,2333_1,"Obviously one of the most important features of any computer is the ""human interface.",human interface,69--84,0
1,1805_1,Good for every day computing and web browsing.,every day computing,9--28,1
2,2782_2,while the keyboard itself is alright[comma] the plate around it is cheap plastic and makes a hollow sound when using the mouse command buttons.,mouse command buttons,115--136,-1
3,1385_0,Again[comma] the same problem[comma] the right speaker did not work.,right speaker,29--42,-1
4,1423_0,My problem was with DELL Customer Service.,DELL Customer Service,20--41,-1
5,104_0,Is this partially due to the fact that it is running Windows Vista?,Windows Vista,53--66,0
6,398_0,Accordingly[comma] I have decided to NEVER purchase another HP product (my five year old Compaq) lasted 5-years before the hard drive crashed.,hard drive,117--127,-1
7,2766_1,I did think it had a camera because that was one of my requirements[comma] but forgot to check in the specifications on this one before I purchased.,specifications,96--110,0
8,2808_5,Cords coming out the right for power plus cords coming out front for headphones/mic plus network connection on left make for a very messy setup with cords going every direction.,setup,138--143,-1
9,1727_0,The most recent being that my Safari internet browser is freaking out on me[comma] but I have just been using firefox instead.,Safari internet browser,30--53,-1


In [52]:
data.loc[:,[' text']]

,text
0,"Obviously one of the most important features of any computer is the ""human interface."
1,Good for every day computing and web browsing.
2,while the keyboard itself is alright[comma] the plate around it is cheap plastic and makes a hollow sound when using the mouse command buttons.
3,Again[comma] the same problem[comma] the right speaker did not work.
4,My problem was with DELL Customer Service.
5,Is this partially due to the fact that it is running Windows Vista?
6,Accordingly[comma] I have decided to NEVER purchase another HP product (my five year old Compaq) lasted 5-years before the hard drive crashed.
7,I did think it had a camera because that was one of my requirements[comma] but forgot to check in the specifications on this one before I purchased.
8,Cords coming out the right for power plus cords coming out front for headphones/mic plus network connection on left make for a very messy setup with cords going every direction.
9,The most recent being that my Safari internet browser is freaking out on me[comma] but I have just been using firefox instead.


In [49]:
#data2.replace('[comma]',',')
#data2[' text'] = data2[' text'].str.replace("[comma]",",")
#data2[' text'].replace({'[comma]': ','}, inplace=True, regex=True)
#data2[' text'].replace({'[comma]':','}, regex=True)

0       But the st,ff w,s s, h,rrible t, us.                                                                                                                                                                                  
1       T, be ,,,pletely f,ir[,,,,,] the ,nly redee,ing f,,t,r w,s the f,,d[,,,,,] whi,h w,s ,b,ve ,ver,ge[,,,,,] but ,,uldn't ,,ke up f,r ,ll the ,ther defi,ien,ies ,f Te,d,r,.                                             
2       The f,,d is unif,r,ly ex,epti,n,l[,,,,,] with , very ,,p,ble kit,hen whi,h will pr,udly whip up wh,tever y,u feel like e,ting[,,,,,] whether it's ,n the ,enu ,r n,t.                                                 
3       The f,,d is unif,r,ly ex,epti,n,l[,,,,,] with , very ,,p,ble kit,hen whi,h will pr,udly whip up wh,tever y,u feel like e,ting[,,,,,] whether it's ,n the ,enu ,r n,t.                                                 
4       The f,,d is unif,r,ly ex,epti,n,l[,,,,,] with , very ,,p,ble kit,hen whi,h will pr,udly whip up wh,t

In [53]:
data2.loc[:,[' text']]

,text
0,But the staff was so horrible to us.
1,To be completely fair[comma] the only redeeming factor was the food[comma] which was above average[comma] but couldn't make up for all the other deficiencies of Teodora.
2,The food is uniformly exceptional[comma] with a very capable kitchen which will proudly whip up whatever you feel like eating[comma] whether it's on the menu or not.
3,The food is uniformly exceptional[comma] with a very capable kitchen which will proudly whip up whatever you feel like eating[comma] whether it's on the menu or not.
4,The food is uniformly exceptional[comma] with a very capable kitchen which will proudly whip up whatever you feel like eating[comma] whether it's on the menu or not.
5,Not only was the food outstanding[comma] but the little 'perks' were great.
6,Not only was the food outstanding[comma] but the little 'perks' were great.
7,Our agreed favorite is the orrechiete with sausage and chicken (usually the waiters are kind enough to split the dish in half so you get to sample both meats).
8,Our agreed favorite is the orrechiete with sausage and chicken (usually the waiters are kind enough to split the dish in half so you get to sample both meats).
9,Our agreed favorite is the orrechiete with sausage and chicken (usually the waiters are kind enough to split the dish in half so you get to sample both meats).


In [56]:
dlist = []
dlist = data[' text']

0       Obviously one of the most important features of any computer is the "human interface.                                                                                                                                                                                                                        
1       Good for every day computing and web browsing.                                                                                                                                                                                                                                                               
2       while the keyboard itself is alright[comma] the plate around it is cheap plastic and makes a hollow sound when using the mouse command buttons.                                                                                                                                                              
3       Again[comma] the same problem[comma] the right speaker did not

In [57]:
print(dlist)

0       Obviously one of the most important features of any computer is the "human interface.                                                                                                                                                                                                                        
1       Good for every day computing and web browsing.                                                                                                                                                                                                                                                               
2       while the keyboard itself is alright[comma] the plate around it is cheap plastic and makes a hollow sound when using the mouse command buttons.                                                                                                                                                              
3       Again[comma] the same problem[comma] the right speaker did not

In [7]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True